In [91]:
#from tkinter.filedialog import askopenfilename
from sklearn.model_selection import cross_val_score
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
import scipy
from scipy import stats
from scipy import stats
import sklearn
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.mixture import GaussianMixture
#import skbio.stats.composition as comp
import plotly.express as px
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import svm # SVC para classificação (target discreto) e SVR para regressão (traget contínuo)
from sklearn.ensemble import RandomForestClassifier #só classificação
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn import metrics
import GeoSed_stat_funcs as gs
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [92]:
#file = askopenfilename()
df0=pd.read_excel('C:\\Users\\iande\\Documentos\\proyectos\\ML with lumi data\\osl_parnaiba_table.xlsx',sheet_name="by_aliquot")
df0 = df0.drop(columns=['Unnamed: 0','sample','Group'])
df0 = df0.fillna(33.333)
display(df0.head())
print(np.unique(df0.Unit))

,Unit,BOSL1s,BOSLF,BOSLM,BOSLS,IRSL1.2s/BOSL1s,TL110oC,TL110pos,TL325pos
0,Serra Grande,12.811713,38.068407,33.330574,28.601019,19.076664,76.183863,96.2,325.8
1,Serra Grande,12.221442,38.772984,31.830463,29.396553,29.591771,75.658334,96.2,325.8
2,Serra Grande,13.004011,40.739419,32.236592,27.023990,17.354350,77.313586,96.2,325.8
3,Serra Grande,11.314952,39.443369,31.129080,29.427551,27.822570,76.931064,96.2,324.0
4,Serra Grande,3.490668,5.954731,19.595849,74.449419,121.146848,58.377886,98.0,334.8


['Cabecas' 'Longa' 'Motuca' 'Piaui' 'Pimenteiras' 'Poti' 'Sambaiba'
 'Serra Grande']


In [93]:
df0 = df0.replace({'Unit':['Serra Grande','Pimenteiras','Longa','Poti','Piaui','Sambaiba','Motuca']},{'Unit':'0'})
df0 = df0.replace({'Unit':'Cabecas'},{'Unit':'1'})
display(df0)

,Unit,BOSL1s,BOSLF,BOSLM,BOSLS,IRSL1.2s/BOSL1s,TL110oC,TL110pos,TL325pos
0,0,12.811713,38.068407,33.330574,28.601019,19.076664,76.183863,96.2,325.8
1,0,12.221442,38.772984,31.830463,29.396553,29.591771,75.658334,96.2,325.8
2,0,13.004011,40.739419,32.236592,27.023990,17.354350,77.313586,96.2,325.8
3,0,11.314952,39.443369,31.129080,29.427551,27.822570,76.931064,96.2,324.0
4,0,3.490668,5.954731,19.595849,74.449419,121.146848,58.377886,98.0,334.8
...,...,...,...,...,...,...,...,...,...
323,1,18.231015,54.266323,14.712779,31.020898,9.833854,50.422455,96.2,325.8
324,0,11.256368,39.588299,16.897768,43.513934,16.675745,43.886663,94.4,325.8
325,0,8.766445,29.489215,24.054931,46.455854,20.252563,39.454300,94.4,333.0
326,0,7.759487,30.913104,21.734711,47.352185,39.636704,42.566480,96.2,329.4


In [94]:
clf = RandomForestClassifier()
y = np.array(df0['Unit'])
y = y.astype(int)
X = np.array(df0['BOSL1s'])
X = X.reshape(-1,1)
type(X),type(y)

(numpy.ndarray, numpy.ndarray)

In [95]:
kfolds = 10
scores = cross_val_score(clf, X, y, cv = kfolds, scoring = "accuracy") 
#print("k test results", scores)
print("test average", scores.mean())
print("standard deviation", scores.std())

test average 0.734564393939394
standard deviation 0.10496598454500095


In [96]:
indices = list(range(1,len(df0.index)))
k = 10

subset_size = round(len(indices) / k)

subsets = [indices[x:x+subset_size] for x in range(0, len(indices), subset_size)]

kfolds = []
for i in range(k):
    test = subsets[i]
    train = []
    for subset in subsets:
        if subset != test:
            train.append(subset)
    kfolds.append((train, test))
# output:
# kfolds = [([0,1,2,3,4],[5,6,7,8,9]), ([5,6,7,8,9],[0,1,2,3,4])]

In [97]:
def kfoldcv(indices, k):
    
    size = len(indices)
    subset_size = round(size / k)
    
    subsets = [indices[x:x+subset_size] for x in range(0, len(indices), subset_size)]
    kfolds = []
    for i in range(k):
        test = subsets[i]
        train = []
        for subset in subsets:
            if subset != test:
                train.append(subset)
        kfolds.append((train,test))
        
    return kfolds

In [98]:
import random
def kfoldcv(indices, k = 10, seed = 42):
    
    size = len(indices)
    subset_size = round(size / k)
    random.Random(seed).shuffle(indices)
    subsets = [indices[x:x+subset_size] for x in range(0, len(indices), subset_size)]
    kfolds = []
    for i in range(k):
        test = subsets[i]
        train = []
        for subset in subsets:
            if subset != test:
                train.append(subset)
        kfolds.append((train,test))
        
    return kfolds